In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

from scipy.ndimage import binary_erosion
import torch
from torchvision.transforms import functional as TF

import pietorch
# Examples to try: image, gif, brain??

# Poisson image editing examples
## Image (2D)

In [ ]:
img_1_png = Image.open('example_images/mug.png')
img_1 = Image.new("RGB", img_1_png.size, (255, 255, 255))
img_1.paste(img_1_png, mask=img_1_png.getchannel('A'))

img_2 = Image.open('example_images/brick_texture.jpg')

fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
ax[0].imshow(img_1)
ax[1].imshow(img_2)

In [ ]:
target = TF.to_tensor(img_1)

source = TF.resize(TF.to_tensor(img_2), [512, 512])

corner_coord = torch.tensor([144, 100])

mask = torch.ones(source.shape[1:])

fig, ax = plt.subplots(ncols=3, figsize=(15, 5))
ax[0].imshow(torch.movedim(target, 0, -1))
ax[1].imshow(torch.movedim(source, 0, -1))
ax[2].imshow(mask)

In [ ]:
res = pietorch.blend(target, source, torch.zeros(source.shape[1:]), corner_coord, True, channels_dim=0)


plt.imshow(torch.movedim(res, 0, -1))

In [ ]:
recon_diff = torch.abs(res - target)

print(torch.min(recon_diff), '-', torch.mean(recon_diff), '/', torch.median(recon_diff), '-', torch.max(recon_diff))

plt.imshow(torch.movedim(recon_diff, 0, -1))

In [ ]:
res = pietorch.blend(target, source, mask, corner_coord, True, channels_dim=0)


plt.imshow(torch.movedim(res, 0, -1))

In [ ]:
diff = torch.abs(res - target)
print(torch.max(diff))
plt.imshow(torch.movedim(diff, 0, -1))


In [ ]:
import cv2

cv_img_1 = np.array(img_1)
cv_img_2 = np.array(TF.resize(img_2, [512, 512]))
cv_mask = mask.numpy().astype(np.uint8) * 255
cv_centre_coord = (corner_coord.numpy() + np.array(cv_mask.shape) // 2)

cv_res = cv2.seamlessClone(cv_img_2,
                           cv_img_1,
                           cv_mask,
                           tuple(cv_centre_coord[::-1]),
                           cv2.NORMAL_CLONE)

plt.imshow(cv_res)

In [ ]:
cv_diff = np.abs(cv_img_1.astype(int) - cv_res.astype(int))
print(np.max(cv_diff))

plt.imshow(cv_diff)

In [ ]:
cv_res = cv2.seamlessClone(cv_img_2,
                           cv_img_1,
                           cv_mask,
                           tuple(cv_centre_coord[::-1]),
                           cv2.MIXED_CLONE)

plt.imshow(cv_res)

In [ ]:
cv_diff = np.abs(cv_img_1.astype(int) - cv_res.astype(int))
print(np.max(cv_diff))

plt.imshow(cv_diff)

In [ ]:
import imageio

def read_gif(file_path):
    return torch.stack([TF.to_tensor(im) for im in imageio.get_reader(file_path)])

wave = read_gif('example_images/wave.gif')[:, :3]
splat = read_gif('example_images/splat.gif')[:, :3]

In [ ]:
plt.imshow(torch.movedim(wave[0], 0, -1))

In [ ]:
target_gif = wave

source_gif = TF.resize(torch.cat([splat, torch.flip(splat, [0])]), (300, 300))

corner_coord = torch.tensor([4, 100, 100])

mask_gif = torch.ones((source_gif.shape[0], *source_gif.shape[2:]))

In [ ]:
res_gif = pietorch.blend(target_gif, source_gif, mask_gif, corner_coord, True, channels_dim=1)

In [ ]:
%matplotlib notebook
from matplotlib.animation import FuncAnimation

gif = res_gif

anim_fig, anim_ax = plt.subplots()
anim_ax.set_axis_off()
img_ax = plt.imshow(torch.movedim(gif[0], 0, -1))

def animate_gif(frame_num):
    img_ax.set_data(torch.movedim(gif[frame_num], 0, -1))
    return img_ax

anim =  FuncAnimation(anim_fig, animate_gif, frames=len(gif), interval=15)
plt.show()

In [ ]:
anim.save('example_images/blended.gif', fps=30)